## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

# 1) Création d'un DataFrame des 35 villes avec coordonnées géographiques

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from datetime import datetime

In [2]:
cities =["Mont-Saint-Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

### Test sur une ville 

In [3]:
r=requests.get('https://nominatim.openstreetmap.org/?addressdetails=1&q=Mont-Saint-Michel&countrycodes=france&format=json')
r.json()[0]

{'place_id': 151486647,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 211285890,
 'boundingbox': ['48.6349172', '48.637031', '-1.5133292', '-1.5094796'],
 'lat': '48.6359541',
 'lon': '-1.511459954959514',
 'display_name': 'Mont Saint-Michel, Le Mont-Saint-Michel, Avranches, Manche, Normandie, France métropolitaine, 50170, France',
 'class': 'place',
 'type': 'islet',
 'importance': 0.755436556781574,
 'address': {'place': 'Mont Saint-Michel',
  'city': 'Le Mont-Saint-Michel',
  'municipality': 'Avranches',
  'county': 'Manche',
  'state': 'Normandie',
  'region': 'France métropolitaine',
  'postcode': '50170',
  'country': 'France',
  'country_code': 'fr'}}

In [4]:
print(r.json()[0]['lat'])
print(r.json()[0]['lon'])

48.6359541
-1.511459954959514


### Récupération des coordonnées pour toutes les villes

In [5]:
columns = ['id', 'name', 'latitude', 'longitude']
data = []

for i, j in enumerate(cities):
    params = {"format": 'json', "limit": 1}
    r=requests.get('https://nominatim.openstreetmap.org/?addressdetails=1&q=' +str(j) +'&countrycodes=france&format=json&limit=1', params=params)
    liste =[i, j, r.json()[0]['lat'], r.json()[0]['lon']]
    data.append(liste)
    
df_cities = pd.DataFrame(data=data, columns=columns)

In [6]:
df_cities.head()

,id,name,latitude,longitude
0,0,Mont-Saint-Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


Transformation de latitude et longitude en numeric

In [7]:
df_cities.dtypes

id            int64
name         object
latitude     object
longitude    object
dtype: object

In [8]:
df_cities[["latitude", "longitude"]] = df_cities[["latitude", "longitude"]].apply(pd.to_numeric)

In [9]:
df_cities.dtypes

id             int64
name          object
latitude     float64
longitude    float64
dtype: object

# 2) Création d'un DataFrame contenant pour chaque ville les données météorologiques que nous souhaitons étudier

### Test sur une ville

In [10]:
api_key='25143b0395a1953fbdcfd568cbd5c928'

In [11]:
para={'lat':df_cities.loc[0,'latitude'],
      'lon':df_cities.loc[0,'longitude'],
      'appid':api_key,
      'exclude': 'current,hourly,minutely',
      'units':'metric',}
r=requests.get('https://api.openweathermap.org/data/2.5/onecall', params=para).json()


In [12]:
print(r['daily'][0]['temp']['day'])
print(r['daily'][0]['humidity'])
print(r['daily'][0]['pop'])
print(r['daily'][0]['weather'][0]['main'])
print(datetime.utcfromtimestamp(r['daily'][0]['dt']).strftime('%Y-%m-%d'))

4.27
71
0
Clouds
2021-12-21


### On récupère pour chaque ville les données météorologique sur les sept prochains jours

In [13]:
liste2=[]
dic = {"city_id" : [],
       "date" : [],
      "temperature" : [],
      "humidity" : [],
      "precipitation_probability" : [],
      "weather" : []}
for i in range(len(cities)):
    para={'lat':df_cities.loc[i,'latitude'],
          'lon':df_cities.loc[i,'longitude'],
          'appid':api_key,
          'exclude': 'current,hourly,minutely',
          'units':'metric',}
    r=requests.get('https://api.openweathermap.org/data/2.5/onecall', params=para).json()
    for j in range(8):
            dic["city_id"].append(df_cities.loc[i, 'id'])
            dic["temperature"].append(r['daily'][j]['temp']['day'])
            dic["humidity"].append(r['daily'][j]['humidity'])
            dic["precipitation_probability"].append(r['daily'][j]['pop'])
            dic["weather"].append(r['daily'][j]['weather'][0]['main'])
            dic["date"].append(datetime.utcfromtimestamp(r['daily'][j]['dt']).strftime('%Y-%m-%d'))
        
df_weather=pd.DataFrame.from_dict(dic)

In [14]:
df_weather.head(10)

,city_id,date,temperature,humidity,precipitation_probability,weather
0,0,2021-12-21,4.27,71,0.00,Clouds
1,0,2021-12-22,5.60,51,0.00,Clouds
2,0,2021-12-23,9.42,90,0.05,Clouds
3,0,2021-12-24,10.85,87,1.00,Rain
4,0,2021-12-25,11.21,82,1.00,Rain
5,0,2021-12-26,11.80,79,1.00,Rain
6,0,2021-12-27,8.00,73,1.00,Rain
7,0,2021-12-28,7.03,80,0.32,Rain
8,1,2021-12-21,3.86,75,0.00,Clouds
9,1,2021-12-22,5.33,55,0.00,Clouds


In [15]:
#Jointure entre nos deux DataFrame
df_cities_weather = df_cities.merge(df_weather, right_on ='city_id', left_on='id').drop('city_id', axis=1)
df_cities_weather.head(10)

,id,name,latitude,longitude,date,temperature,humidity,precipitation_probability,weather
0,0,Mont-Saint-Michel,48.635954,-1.511460,2021-12-21,4.27,71,0.00,Clouds
1,0,Mont-Saint-Michel,48.635954,-1.511460,2021-12-22,5.60,51,0.00,Clouds
2,0,Mont-Saint-Michel,48.635954,-1.511460,2021-12-23,9.42,90,0.05,Clouds
3,0,Mont-Saint-Michel,48.635954,-1.511460,2021-12-24,10.85,87,1.00,Rain
4,0,Mont-Saint-Michel,48.635954,-1.511460,2021-12-25,11.21,82,1.00,Rain
5,0,Mont-Saint-Michel,48.635954,-1.511460,2021-12-26,11.80,79,1.00,Rain
6,0,Mont-Saint-Michel,48.635954,-1.511460,2021-12-27,8.00,73,1.00,Rain
7,0,Mont-Saint-Michel,48.635954,-1.511460,2021-12-28,7.03,80,0.32,Rain
8,1,St Malo,48.649518,-2.026041,2021-12-21,3.86,75,0.00,Clouds
9,1,St Malo,48.649518,-2.026041,2021-12-22,5.33,55,0.00,Clouds


Enregistrement du DataFrame en format csv

In [16]:
df_cities_weather.to_csv('cities_weather.csv', index=False)